# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated review score.

For that purpose, we will create one single data table containing **all our orders with some engineered statistics for them as additional columns.**

👉 Our goal is to create a DataFrame with the following features:

*- it will bcome in very handy for our modeling phase -*


| feature_name 	| type 	| description 	|
|:---	|:---:	|:---	|
| `order_id` 	| str 	| the id of the order 	|
| `wait_time` 	| float 	| the number of days between order_date and delivered_date 	|
| `expected_wait_time` 	| float 	| the number of days between order_date and estimated_delivery_date 	|
| `delay_vs_expected` 	| float 	| if the actual delivery date is later than the estimated delivery date, returns the number of days between the two dates, otherwise return 0 	|
| `order_status` 	| str 	| the status of the order 	|
| `dim_is_five_star` 	| int 	| 1 if the order received a five-star review, 0 otherwise 	|
| `dim_is_one_star` 	| int 	| 1 if the order received a one_star, 0 otherwise 	|
| `review_score` 	| int 	| from 1 to 5 	|
| `number_of_products` 	| int 	| number of products that the order contains 	|
| `number_of_sellers` 	| int 	| number of sellers involved in the order 	|
| `price` 	| float 	| total price of the order paid by customer 	|
| `freight_value` 	| float 	| value of the freight paid by customer 	|
| `distance_customer_seller` 	| float 	| the distance in km between customer and seller (optional) 	|  
  
⚠️ We also want to filter out "non-delivered" orders, unless explicitly specified, otherwise we cannot compute the potential delays.

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame.

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

🔥 Notebook best practices (must-read) 👇

<details>
    <summary> - <i>click here</i> - </summary>

From now on, exploratory notebooks are going to become pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double-click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsable Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





In [107]:
# Auto reload imported module everytime a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
# Import usual modules
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [109]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()
matching_table = olist.get_matching_table()

## Code `order.py`

In [110]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

### `get_wait_time`
    Return a Dataframe with:
           order_id, wait_time, expected_wait_time, delay_vs_expected, order_status

*- Hints -*
- Don't forget to convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
- Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are 

🎁 We give you the pseudo-code below 👇 for this first method:

> 1. Inspect the `orders` dataframe
2. Filter the dataframe on `delivered orders`
3. Handle `datetime`
4. Compute `wait_time`
5. Compute `expected_wait_time`
6. Compute `delay_vs_expected`
7. Check the new dataframe 
8. Once you are satisfied with your code, you can carefully copy-paste it from the notebook to to `olist/order.py`

In [111]:
orders.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


In [112]:
# YOUR CODE HERE

orders_delivered = orders[orders["order_status"] == "delivered"]
## Handle datetime
liste = ["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date" , "order_estimated_delivery_date" ]
test  = orders_delivered.loc[:,liste].apply(pd.to_datetime)
orders_delivered[liste] = test
orders_delivered["wait_time"] = orders_delivered["order_delivered_carrier_date"] - orders_delivered["order_purchase_timestamp"]
orders_delivered["expected_wait_time"] =orders_delivered["order_estimated_delivery_date"] - orders_delivered["order_purchase_timestamp"]
delay_vs_expected =  orders_delivered["wait_time"] -orders_delivered["expected_wait_time"]
delay_vs_expected[delay_vs_expected < datetime.timedelta(minutes=0)]  = delay_vs_expected[delay_vs_expected < datetime.timedelta(minutes=0)] + delay_vs_expected[delay_vs_expected < datetime.timedelta(minutes=0)] * - 1
orders_delivered["delay_vs_expected"] = delay_vs_expected
TT = orders_delivered[["order_id", "wait_time", "expected_wait_time", "delay_vs_expected", "order_status"]]



/Users/selmalopez/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/var/folders/5t/r6_mkf0x1zl9f1rvdldpw7pr0000gn/T/ipykernel_77196/3913996880.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_delivered["wait_time"] = orders_delivered["order_delivered_carrier_date"] - orders_delivered["order_purchase_timestamp"]
/var/folders/5t/r6_mkf0x1zl9f1rvdldpw7pr0000gn/T/ipykernel_77196/3913996

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🙏 Now, test it by running the following cell 👇 

In [113]:
# Test your code here
from olist.order import Order
Order().get_wait_time()

/Users/selmalopez/code/selmalopez/data-challenges/04-Decision-Science/olist/order.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_delivered["wait_time"] = orders_delivered["order_delivered_carrier_date"] - orders_delivered["order_purchase_timestamp"]
/Users/selmalopez/code/selmalopez/data-challenges/04-Decision-Science/olist/order.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_delivered["expected_wait_time"] =orders_delivered["order_estimated_delivery_date"] - orders_delivered["orde

### `get_review_score`
     Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

👉 Load the `reviews`

In [114]:
# YOUR CODE HERE
reviews = data["order_reviews"]
reviews

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,NaN,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13


👉 Let's create two functions `dim_five_star` and `dim_one_star`  
    We will apply them  `element_wise` to the `review_score` column in the next cell below.

In [115]:
def dim_five_star(d):
    if d == 5:
        return 1
    return 0

def dim_one_star(d):
    if d == 1:
        return 1
    return 0

👉 Use these functions to create two boolean features `dim_is_five_star` and `dim_is_one_star`

In [117]:
# YOUR CODE HERE
reviews["dim_is_five_star"] = list(map(dim_five_star,reviews["review_score"]))
reviews["dim_is_one_star"] = list(map(dim_one_star,reviews["review_score"]))

Once again, 

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🙏 Now, test it by running the following cell 👇 

In [163]:
# Test your code here
from olist.order import Order
Order().get_review_score()

#### Check your code

In [92]:
from nbresult import ChallengeResult

result = ChallengeResult('reviews',
    dim_five_star=dim_five_star(5),
    dim_not_five_star=dim_five_star(3),
    dim_one_star=dim_one_star(1),
    dim_not_one_star=dim_one_star(2)
)
result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/selmalopez/.pyenv/versions/lewagon_current/bin/python3
cachedir: .pytest_cache
rootdir: /Users/selmalopez/code/selmalopez/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: dash-2.0.0, anyio-3.3.2
collecting ... collected 4 items

tests/test_reviews.py::TestReviews::test_dim_five_star PASSED            [ 25%]
tests/test_reviews.py::TestReviews::test_dim_not_five_star PASSED        [ 50%]
tests/test_reviews.py::TestReviews::test_dim_not_one_star PASSED         [ 75%]
tests/test_reviews.py::TestReviews::test_dim_one_star PASSED             [100%]

============================== 4 passed in 0.07s ===============================


💯 You can commit your code:

git add tests/reviews.pickle

git commit -m 'Completed reviews step'

git push origin master


### `get_number_products`:
     Returns a DataFrame with:
        order_id, number_of_products (total number of products per order)

In [156]:
# YOUR CODE HERE
p_df = data["order_items"].copy()
p_df = p_df[["order_id","product_id"]]
p_df = p_df.groupby('order_id').count()
p_df = p_df.rename(columns={'product_id':'number_of_products'}).reset_index()
p_df

,order_id,number_of_products
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


👉 Same routine: 
* check your dataframe, 
* commit your code to `olist/order.py`
* and check that it truly works.

In [157]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_number_products()

NameError: name 'data' is not defined

### `get_number_sellers`:
     Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)
        
<details>
    <summary>- <i>Hint</i> -</summary>

`pd.Series.nunique()`
</details>

## YOUR CODE HERE


In [144]:
d_df = data["order_items"].copy()
#d_df = d_df[["order_id","number_of_sellers"]]
d_df = d_df.groupby('order_id').nunique()
d_df['seller_id']
d_df.value_counts('seller_id')
#d_df = d_df.groupby('order_id').count()

seller_id
1    97388
2     1219
3       54
4        3
5        2
dtype: int64

In [145]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_number_sellers()

### `get_price_and_freight`
     Returns a DataFrame with:
        order_id, price, freight_value

<details>
    <summary>- <i>Hint -</i></summary>

`pd.Series.agg()` allows you to apply one transformation method per column of your groupby object
</details>

In [155]:
# YOUR CODE HERE
s_df = data["order_items"].copy()
s_df = s_df[['order_id','price','freight_value']]
s_df = s_df.groupby('order_id').agg({'price':'sum','freight_value': 'sum'}).reset_index()
s_df.head()

,order_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14


In [159]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_price_and_freight()

### `get_distance_seller_customer` 
**(OPTIONAL - Try  to code this function only after finishing today's challenges - Skip to next section)**

    Returns a Dataframe with:
        [order_id, distance_seller_customer] 
               (the distance in km between customer and seller)

💡Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

In [ ]:
# YOUR CODE HERE

👀 Check your new dataframe and commit your code to olist/order.py when it works. 

In [46]:
# YOUR CODE HERE

# Test your newly coded module

❓ Time to code `get_training_data` making use of your previous coded methods.

In [164]:
from olist.order import Order
from nbresult import ChallengeResult
data = Order().get_training_data()
result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())

/Users/selmalopez/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/selmalopez/code/selmalopez/data-challenges/04-Decision-Science/olist/order.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_delivered["wait_time"] = orders_delivered["order_delivered_carrier_date"] - orders_delivered["order_purchase_timestamp"]
/Users/selmalopez/code/selmalopez/data-challenges/04-Decision-Sci

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/selmalopez/.pyenv/versions/lewagon_current/bin/python3
cachedir: .pytest_cache
rootdir: /Users/selmalopez/code/selmalopez/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: dash-2.0.0, anyio-3.3.2
collecting ... collected 2 items

tests/test_training.py::TestTraining::test_training_data_columns FAILED  [ 50%]
tests/test_training.py::TestTraining::test_training_data_shape FAILED    [100%]

=================================== FAILURES ===================================
___________________ TestTraining.test_training_data_columns ____________________

self = <tests.test_training.TestTraining testMethod=test_training_data_columns>

    def test_training_data_columns(self):
        columns = ['delay_vs_expected',
                   'dim_is_five_star',
                   'dim_is_one_star',
             

🏁 Congratulations! 

⌛️ Commit and push your notebook before starting the next challenge.